In [2]:
import pandas as pd
import numpy as np

source price data <https://transparency.entsoe.eu/transmission-domain/r2/dayAheadPrices/show?name=&defaultValue=true&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=16.02.2024+00:00|CET|DAY&biddingZone.values=CTY|10YNO-0--------C!BZN|10YNO-1--------2&resolution.values=PT60M&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)>

In [103]:

df_21 = pd.read_csv('no2_2021.csv')
df_22 = pd.read_csv('no2_2022.csv')
df_23 = pd.read_csv('no2_2023.csv')
pdList = [df_21, df_22 ]
df = pd.concat(pdList)
df = df.reset_index().drop('index', axis = 1)
df['date'] =  df['MTU (UTC)'].str[:16]
df = df.rename(columns={'Day-ahead Price [EUR/MWh]':'price_no2'})[['date','price_no2']]
df_23['date'] =  df_23['MTU (CET/CEST)'].str[:16]
df_23 = df_23.rename(columns={'Day-ahead Price [EUR/MWh]':'price_no2'})[['date','price_no2']]

df = pd.concat([df,df_23])

df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y %H:%M')
# Set year as a column
df['year'] = df['date'].dt.year
# change priec_no2 to float
df['price_no2'] = df['price_no2'].astype(float)


In [65]:
df.groupby('year').mean('price_no2')

,price_no2
year,
2021,75.107840
2022,211.277426
2023,79.446628


# Pull in windproduction data in NO2

-   source: <https://elhub.no/data/apnedata/#production_per_group_mba_hour>

In [104]:
production = pd.read_csv('production_NO.csv')
# make column names to small letters
production.columns = production.columns.str.lower()
production = production[(production['price_area'] == "NO2") & (production['production_group'] == "Wind unspecified")]
production = production[['start_time','volume_kwh']]
# Convert the column to string
production['start_time'] = production['start_time'].astype(str)
# Remove the timezone part by keeping only the characters before the '+'
# Have extracted data from entso-e in same timeformat as reported by elhub
production['start_time'] = production['start_time'].str.slice(stop=-6)

# Convert the resulting string to datetime
production['start_time'] = pd.to_datetime(production['start_time'])
df = pd.merge(df, production, left_on='date', right_on='start_time', how='left')
df['volume_mwh'] = df['volume_kwh'] *0.001 # compensate for volume in kwh 
df['revenue_wind'] = df['volume_mwh'] * df['price_no2']


In [115]:
grouped_df = df.groupby('year').agg({
    'revenue_wind': 'sum',
    'volume_mwh': 'sum',
    'price_no2': 'mean'
}).reset_index()
grouped_df['wind_captured'] = grouped_df['revenue_wind'] / grouped_df['volume_mwh']
grouped_df['verdifaktor'] = grouped_df['wind_captured'] / grouped_df['price_no2']

In [121]:
#rename price_no2 to mean_price_no2
grouped_df.rename(columns={'price_no2':'mean_price_no2'}, inplace = True)


In [126]:
grouped_df[['year','mean_price_no2','wind_captured','verdifaktor']]

,year,mean_price_no2,wind_captured,verdifaktor
0,2021,75.105980,76.588413,1.019738
1,2022,211.264206,188.678098,0.893091
2,2023,79.433936,71.934591,0.905590


# SE3 and nuclear
- Did not find production data for whole of 2021 so only 2022 and 2023

In [57]:
df_22 = pd.read_csv('se/price_se3_2022.csv')
df_23 = pd.read_csv('se/price_se3_2023.csv')
pdList = [ df_22, df_23 ]
price_se3 = pd.concat(pdList)
price_se3 = price_se3.reset_index().drop('index', axis = 1)

df_22 = pd.read_csv('se/production_se3_2022.csv')
df_23 = pd.read_csv('se/production_se3_2023.csv')
pdList = [df_22, df_23 ]
production_se3 = pd.concat(pdList)
production_se3 = production_se3.reset_index().drop('index', axis = 1)
# Clean names
production_se3.columns = [col.split('-')[0].strip().lower().replace(' ', '_') for col in production_se3.columns]
#clean up date format
production_se3['mtu'] = production_se3['mtu'].str.replace(' \(CET/CEST\)', '', regex=True)
# Merge production and price data

df =pd.merge(price_se3, production_se3, left_on='MTU (CET/CEST)', right_on='mtu', how='left')
# Remove 4 observations with ti high nuclear production
df = df[df['nuclear']<10000]
# Collect year from mtu column
df['year'] = df['mtu'].str.extract(r'\d{2}\.\d{2}\.(\d{4})')
# rename column "Day-ahead Price [EUR/MWh]" to price

df = df.rename(columns={'Day-ahead Price [EUR/MWh]':'price'})

#calculate hourly revenye for nuclear
df['nuclear_revenue'] = df['nuclear'] * df['price']

Installed nuclear capacity = 6944mwh

<https://world-nuclear.org/information-library/country-profiles/countries-o-s/sweden.aspx>

In [60]:
grouped_df = df.groupby('year').agg({
    'nuclear_revenue': 'sum',
    'nuclear': 'sum',
    'price': 'mean',
}).reset_index()
grouped_df['nuclear_captured'] = grouped_df['nuclear_revenue'] / grouped_df['nuclear']
grouped_df['verdifaktor'] = grouped_df['nuclear_captured'] / grouped_df['price']

In [63]:
grouped_df[['year','price','nuclear_captured','verdifaktor']]

,year,price,nuclear_captured,verdifaktor
0,2022,129.244873,125.804979,0.973385
1,2023,51.695039,54.218847,1.048821


In [64]:
group_prod = df.groupby('year').agg({
    'nuclear': 'mean'
}).reset_index()
group_prod['capacity_factor'] = group_prod['nuclear'] / 6944
group_prod

,year,nuclear,capacity_factor
0,2022,5722.148681,0.824042
1,2023,5318.558777,0.765921
